In [62]:
import pandas as pd
import random
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import coint, adfuller
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [63]:
# Get S&P 500 tickers
def get_sp500_tickers(sample_size=200):
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url)[0]
    tickers = table['Symbol'].tolist()
    sample_tickers = random.sample(tickers, sample_size)
    return sample_tickers

In [64]:
# Fetch historical data
def get_historical_data(tickers):
    data = pd.DataFrame()
    for i in tickers:
        stock_data = yf.download(i, start="2022-01-01", end="2024-09-10")
        data[i] = stock_data['Adj Close']
    return data

In [65]:
# Perform the ADF test for stationarity
def adf_test(series):
    result = adfuller(series)
    return result[1]  # Return p-val

In [66]:
# Cointegration test between pairs of stocks
def cointegration_test(data):
    pairs = []
    results = []
    
    tickers = data.columns
    for i in range(len(tickers)):
        for j in range(i + 1, len(tickers)):
            x = data[tickers[i]].dropna()
            y = data[tickers[j]].dropna()
            
            x, y = x.align(y, join='inner')  # Align x and y by matching labels (dates)
            if x.empty or y.empty:
                continue
            
            try:
                score, p_value, _ = coint(x, y)
            except ValueError:
                continue
            
            ratio = x / y
            adf_p_value = adf_test(ratio)
            
            pairs.append((tickers[i], tickers[j]))
            results.append((tickers[i], tickers[j], score, p_value, adf_p_value))
    
    results_df = pd.DataFrame(results, columns=['Stock 1', 'Stock 2', 'Cointegration Score', 'Cointegration p-value', 'ADF p-value'])
    return results_df

In [67]:
# Calculate z-score for the ratio
def calculate_zscore(ratio):
    mean = ratio.rolling(window=30).mean()
    std = ratio.rolling(window=30).std()
    zscore = (ratio - mean) / std
    return zscore

In [68]:
# Feature engineering: calculate moving averages and z-scores
def generate_features(ratio):
    ratio = ratio.dropna()
    features = pd.DataFrame()
    
    features['30d_ma'] = ratio.rolling(window=30).mean()
    features['5d_ma'] = ratio.rolling(window=5).mean()
    features['z_score'] = calculate_zscore(ratio)
    
    # Drop rows with NaN values after rolling windows
    features.dropna(inplace=True)
    
    return features

In [69]:
# Get S&P 500 tickers and historical data
ticks = get_sp500_tickers()
historical_data = get_historical_data(ticks)

# Cointegration test
results_df = cointegration_test(historical_data)

# Filter pairs with the lowest ADF p-values
filtered_results = results_df[results_df['ADF p-value'] < 0.05]

# Sort pairs by cointegration score to find the most cointegrated pair
sorted_results = filtered_results.sort_values(by='Cointegration Score', ascending=True)

# Get the top cointegrated pair
top_pair = sorted_results.iloc[0]
stock1, stock2 = top_pair['Stock 1'], top_pair['Stock 2']

# Calculate the ratio for the top pair
ratio = historical_data[stock1] / historical_data[stock2]

# Generate features for the ratio
features = generate_features(ratio)

# Define target signals based on z-score thresholds
entry_threshold = 1
exit_threshold = 0

features['target'] = np.where(features['z_score'] > entry_threshold, -1,
                              np.where(features['z_score'] < -entry_threshold, 1, 0))

# Split data into train and test sets
X = features[['30d_ma', '5d_ma']]
y = features['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False) # shuffle = False to prevent lookahead bias

# Train a Random Forest classifier
# model = RandomForestClassifier()
model = SVC(kernel='rbf', C=1.0)
model.fit(X_train, y_train)

# Predict signals on the test set
y_pred = model.predict(X_test)

print("historica_data:", historical_data.head())
print("results_df:", results_df.head())
print("filtered_results:", filtered_results.head())
print("sorted_results:", sorted_results)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

historica_data:                    ROK         ZBH         WST       AMCR        BIIB  \
Date                                                                    
2022-01-03  323.161804  122.870041  443.033966  10.465219  244.139999   
2022-01-04  326.434753  124.116531  426.223511  10.632592  241.729996   
2022-01-05  320.973541  123.707397  401.355499  10.650210  239.270004   
2022-01-06  318.832794  122.679749  405.101074  10.562118  237.300003   
2022-01-07  311.954010  121.937561  387.942871  10.676638  232.600006   

                   STZ        KKR        ANSS        CPAY        HWM  ...  \
Date                                                                  ...   
2022-01-03  243.034622  71.525925  395.489990  231.110001  32.043404  ...   
2022-01-04  244.745743  72.294495  391.100006  237.250000  33.232758  ...   
2022-01-05  243.553726  67.109062  376.940002  237.300003  33.024624  ...   
2022-01-06  235.315430  68.675385  368.880005  239.259995  33.351696  ...   
2022-01-07

In [70]:
# Backtest using ML signals
def backtest_ml_strategy(stock1_prices, stock2_prices, ml_predictions):
    positions = []
    returns = []
    
    position = None
    entry_stock1_price = None
    entry_stock2_price = None

    for i in range(len(ml_predictions)):
        if ml_predictions[i] == 1 and position is None:
            # Enter long (Long Stock 1, Short Stock 2)
            entry_stock1_price = stock1_prices.iloc[i]
            entry_stock2_price = stock2_prices.iloc[i]
            position = 'long'
            positions.append(('long', i))

        elif ml_predictions[i] == -1 and position is None:
            # Enter short (Short Stock 1, Long Stock 2)
            entry_stock1_price = stock1_prices.iloc[i]
            entry_stock2_price = stock2_prices.iloc[i]
            position = 'short'
            positions.append(('short', i))

        # Exit on model prediction or if zscore comes close to 0 (another exit rule)
        elif (ml_predictions[i] == 0 or abs(features['z_score'].iloc[i]) < 0.1) and position is not None:
            # Exit position
            exit_stock1_price = stock1_prices.iloc[i]
            exit_stock2_price = stock2_prices.iloc[i]
            
            if position == 'long':
                # Return for a long position
                returns.append(((exit_stock1_price - entry_stock1_price)/entry_stock1_price) + ((entry_stock2_price - exit_stock2_price)/entry_stock2_price))
            elif position == 'short':
                # Return for a short position
                returns.append(((entry_stock1_price - exit_stock1_price)/entry_stock1_price) - ((exit_stock2_price - entry_stock2_price)/entry_stock2_price))
            
            # Reset position
            position = None

    return positions, returns

# Adjust stock prices for the test period
stock1_prices_test = historical_data[stock1][-len(y_test):]
stock2_prices_test = historical_data[stock2][-len(y_test):]

# Call the backtest function with appropriate inputs
positions, returns = backtest_ml_strategy(stock1_prices_test, stock2_prices_test, y_pred)

# Converting returns to a pandas Series
returns = pd.Series(returns)

# Calculating risk-free rate
risk_free_rate = 0.02  # Assuming a 5% annual risk-free rate
daily_risk_free_rate = (1 + risk_free_rate) ** (1 / 252) - 1  # Adjusted for daily returns

# Excess returns (returns - daily risk-free rate)
excess_returns = returns - daily_risk_free_rate

# Mean of excess returns
mean_excess_return = excess_returns.mean()

# Standard deviation of returns (volatility)
volatility = returns.std()

sharpe_ratio = mean_excess_return / volatility

# Print positions and returns
print("Positions:", positions)
print("Returns:", returns)
print("Total return:", sum(returns))
print(f"Sharpe Ratio: {sharpe_ratio}")

Positions: [('short', 0), ('long', 2), ('long', 6), ('short', 13), ('long', 19), ('long', 21), ('short', 25), ('short', 27), ('long', 30), ('short', 33), ('long', 35), ('long', 38), ('short', 43), ('long', 45), ('short', 50), ('long', 58), ('long', 63), ('long', 65), ('short', 67), ('short', 71), ('long', 78), ('short', 82), ('short', 86), ('long', 89), ('long', 95), ('short', 98), ('long', 105), ('short', 109), ('long', 113), ('long', 116), ('short', 121), ('short', 123), ('short', 125)]
Returns: [np.float64(-0.010988025939963014), np.float64(0.02049157892381879), np.float64(-0.004486379962777375), np.float64(-0.04482435883050096), np.float64(-0.0042273809064677335), np.float64(0.02581395754670999), np.float64(0.035015500991991105), np.float64(0.019635208539303464), np.float64(-0.00012851389399853863), np.float64(-0.007702439587359616), np.float64(0.03805392747524611), np.float64(-0.03414719307580683), np.float64(0.045457622839823596), np.float64(-0.009251209002613808), np.float64(-0.